# Semantic Kernel với Tích hợp Máy chủ OpenBnB MCP

Notebook này minh họa cách sử dụng Semantic Kernel với máy chủ OpenBnB MCP thực tế để tìm kiếm chỗ ở Airbnb thực sự bằng cách sử dụng MCPStdioPlugin. Để truy cập LLM, nó sử dụng Azure AI Foundry. Để thiết lập các biến môi trường của bạn, bạn có thể làm theo [Bài học Cài đặt](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tạo kết nối Plugin MCP

Chúng ta sẽ kết nối với [máy chủ MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) bằng cách sử dụng MCPStdioPlugin. Máy chủ này cung cấp chức năng tìm kiếm Airbnb thông qua gói @openbnb/mcp-server-airbnb.


## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng Azure AI Foundry để truy cập LLM. Hãy đảm bảo rằng các biến môi trường của bạn đã được thiết lập đúng cách.


## Cấu hình Môi trường

Cấu hình các thiết lập Azure OpenAI. Đảm bảo rằng bạn đã thiết lập các biến môi trường sau:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Hiểu về Tích hợp MCP của OpenBnB

Notebook này kết nối với **máy chủ MCP thực của OpenBnB**, cung cấp chức năng tìm kiếm Airbnb thực tế.

### Cách hoạt động:

1. **MCPStdioPlugin**: Sử dụng giao tiếp đầu vào/đầu ra tiêu chuẩn với máy chủ MCP  
2. **Gói NPM thực**: Tải xuống và chạy `@openbnb/mcp-server-airbnb` thông qua npx  
3. **Dữ liệu trực tiếp**: Trả về dữ liệu thực tế về các bất động sản Airbnb từ API của họ  
4. **Khám phá chức năng**: Agent tự động phát hiện các chức năng có sẵn từ máy chủ MCP  

### Các chức năng có sẵn:

Máy chủ MCP của OpenBnB thường cung cấp:  
- **search_listings** - Tìm kiếm các bất động sản Airbnb theo địa điểm và tiêu chí  
- **get_listing_details** - Lấy thông tin chi tiết về các bất động sản cụ thể  
- **check_availability** - Kiểm tra tình trạng sẵn có cho các ngày cụ thể  
- **get_reviews** - Lấy đánh giá cho các bất động sản  
- **get_host_info** - Lấy thông tin về chủ sở hữu bất động sản  

### Yêu cầu trước:

- **Node.js** được cài đặt trên hệ thống của bạn  
- **Kết nối Internet** để tải gói máy chủ MCP  
- **NPX** khả dụng (đi kèm với Node.js)  

### Kiểm tra kết nối:

Bạn có thể kiểm tra máy chủ MCP thủ công bằng cách chạy:  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

Lệnh này sẽ tải xuống và khởi động máy chủ MCP của OpenBnB, sau đó Semantic Kernel sẽ kết nối để lấy dữ liệu Airbnb thực tế.  


## Chạy Agent với OpenBnB MCP Server

Bây giờ chúng ta sẽ chạy AI Agent kết nối với OpenBnB MCP server để tìm kiếm chỗ ở thực tế trên Airbnb tại Stockholm cho 2 người lớn và 1 trẻ em. Bạn có thể thay đổi danh sách `user_inputs` để điều chỉnh tiêu chí tìm kiếm.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Tóm tắt
Chúc mừng! Bạn đã thành công xây dựng một tác nhân AI tích hợp với tìm kiếm chỗ ở thực tế bằng cách sử dụng Giao thức Ngữ cảnh Mô hình (MCP):

## Công nghệ đã sử dụng:
- Semantic Kernel - Để xây dựng các tác nhân thông minh với Azure OpenAI
- Azure AI Foundry - Cho khả năng LLM và hoàn thành hội thoại
- MCP (Model Context Protocol) - Để tích hợp công cụ theo chuẩn hóa
- OpenBnB MCP Server - Cho chức năng tìm kiếm Airbnb thực tế
- Node.js/NPX - Để chạy máy chủ MCP bên ngoài

## Những điều bạn đã học:
- Tích hợp MCP: Kết nối các tác nhân Semantic Kernel với máy chủ MCP bên ngoài
- Truy cập dữ liệu thời gian thực: Tìm kiếm các tài sản Airbnb thực tế thông qua API trực tiếp
- Giao tiếp qua giao thức: Sử dụng giao tiếp stdio giữa tác nhân và máy chủ MCP
- Khám phá chức năng: Tự động phát hiện các chức năng có sẵn từ máy chủ MCP
- Phản hồi theo luồng: Ghi lại và lưu trữ các cuộc gọi chức năng trong thời gian thực
- Kết xuất HTML: Định dạng phản hồi của tác nhân với bảng được thiết kế và hiển thị tương tác

## Bước tiếp theo:
- Tích hợp thêm các máy chủ MCP (thời tiết, chuyến bay, nhà hàng)
- Xây dựng hệ thống đa tác nhân kết hợp MCP và giao thức A2A
- Tạo máy chủ MCP tùy chỉnh cho các nguồn dữ liệu của riêng bạn
- Triển khai bộ nhớ hội thoại liên tục qua các phiên làm việc
- Đưa tác nhân lên Azure Functions với sự điều phối máy chủ MCP
- Thêm khả năng xác thực người dùng và đặt chỗ



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
